### 🧠 1. Detekce objektů pomocí DETR (Visual Transformer)

In [ ]:
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image, ImageDraw
import requests
from io import BytesIO

# Načtení modelu DETR
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
model.eval()

# Načtení obrázku
def load_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

# Inference a vykreslení
def detect_objects(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]

    draw = ImageDraw.Draw(image)
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        if score > 0.7:
            box = [round(i, 2) for i in box.tolist()]
            draw.rectangle(box, outline="red", width=2)
            draw.text((box[0], box[1]), f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}", fill="red")
    image.show()

# Ukázka
img_url = "https://ultralytics.com/images/zidane.jpg"
image = load_image(img_url)
detect_objects(image)

### 👶 2. Odhad věku z obličeje pomocí modelu z Hugging Face

In [ ]:
import requests
from PIL import Image
from io import BytesIO

from transformers import ViTFeatureExtractor, ViTForImageClassification

# Get example image from official fairface repo + read it in as an image
r = requests.get('https://github.com/dchen236/FairFace/blob/master/detected_faces/race_Asian_face0.jpg?raw=true')
im = Image.open(BytesIO(r.content))

# Init model, transforms
model = ViTForImageClassification.from_pretrained('nateraw/vit-age-classifier')
transforms = ViTFeatureExtractor.from_pretrained('nateraw/vit-age-classifier')

# Transform our image and pass it through the model
inputs = transforms(im, return_tensors='pt')
output = model(**inputs)

# Predicted Class probabilities
proba = output.logits.softmax(1)

# Predicted Classes
preds = proba.argmax(1)

### ✅ Model: MKgoud/License-Plate-Recognizer
- Architektura: YOLOv8m (pojede i na ubohé GPU s 3GB vRAM)
- Úloha: Detekce a ořez registračních značek z obrázků vozidel
- Formát: .pt model kompatibilní s knihovnou ultralytics

In [ ]:
import torch
from PIL import Image
import cv2
import numpy as np
import requests
from io import BytesIO
from ultralytics import YOLO

# Načtení modelu z lokálního souboru nebo Hugging Face - morsetechlab/yolov11-license-plate-detection
model = YOLO('yolov11x-license-plate.pt')

# Načtení obrázku z URL nebo lokálně
url = "https://upload.wikimedia.org/wikipedia/commons/3/3f/French_vehicle_license_plate.jpg"
response = requests.get(url)
image = Image.open(BytesIO(response.content)).convert("RGB")

# Inference
results = model(image)

# Zobrazení výsledků
results.print()
results.show()  # zobrazí obrázek s bounding boxy

# Uložení výsledku
results.save()  # uloží do runs/detect/

### 🧪 3. Vlastní načítání obrázků a příprava tensoru (bez knihoven typu ImageProcessor)

In [ ]:
from PIL import Image
import torch
import numpy as np

# Parametry pro normalizaci (např. ImageNet)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Funkce pro načtení a zpracování obrázku
def load_and_preprocess_image(path, size=(224, 224)):
    # 1. Načtení obrázku
    image = Image.open(path).convert("RGB")  # převod na RGB

    # 2. Změna velikosti
    image = image.resize(size)

    # 3. Převod na NumPy pole a normalizace do rozsahu [0, 1]
    image_np = np.array(image).astype(np.float32) / 255.0

    # 4. Přeskládání dimenzí (HWC → CHW)
    image_np = image_np.transpose((2, 0, 1))

    # 5. Normalizace podle mean a std
    for i in range(3):
        image_np[i] = (image_np[i] - mean[i]) / std[i]

    # 6. Převod na PyTorch tensor a přidání batch dimenze
    image_tensor = torch.tensor(image_np).unsqueeze(0)

    return image_tensor

# Ukázka použití
image_path = "data/fotka.jpg"  # nahraď vlastní cestou
tensor = load_and_preprocess_image(image_path)
print("Tensor shape:", tensor.shape)
